#### Run the prediction and show the image

In [ ]:
using Mocha

backend = CPUBackend()
init(backend)

#Create a memory data layer in the prediction model that we computation
#sequentially tie to the next image we want to predict on.
mem_data = MemoryDataLayer(name="data", tops=[:data], batch_size=1,
  data=Array[zeros(Float32, 28, 28, 1, 1)])

#Create the network with two conv/pooling layers and two fully
#connected inner product layers
conv_layer = ConvolutionLayer(name="conv1", n_filter=20, kernel=(5,5),
  bottoms=[:data], tops=[:conv1])
pool_layer = PoolingLayer(name="pool1", kernel=(2,2), stride=(2,2),
  bottoms=[:conv1], tops=[:pool1])
conv2_layer = ConvolutionLayer(name="conv2", n_filter=20, kernel=(5,5),
  bottoms=[:pool1], tops=[:conv2])
pool2_layer = PoolingLayer(name="pool2", kernel=(2,2), stride  = (2,2),
  bottoms=[:conv2], tops=[:pool2])
fc1_layer = InnerProductLayer(name="ip1", output_dim=500,
  neuron=Neurons.ReLU(), bottoms=[:pool2], tops=[:ip1])
fc2_layer = InnerProductLayer(name="ip2", output_dim=10,
  bottoms=[:ip1], tops=[:ip2])

#Instead of a SoftmaxLossLayer like the training script in this file we
#have a pure SoftmaxLayer in order to determine the probability of the 
#image compared against the different n classification options
softmax_layer = SoftmaxLayer(name="prob", tops=[:prob], bottoms=[:ip2])

#Build the network
common_layers = [conv_layer, pool_layer, conv2_layer, pool2_layer,
  fc1_layer, fc2_layer]
run_net = Net("imagenet", backend, [mem_data, common_layers..., softmax_layer])

#Load the latest snapshot from the training data
load_snapshot(run_net, "snapshots_bak/snapshot-010000.jld")

#Now load one image and run it through the network
using HDF5, ImageView, Images
fid = h5open("data/test.hdf5", "r")
features = read(fid, "data")
labels = read(fid, "label")

get_layer(run_net, "data").data[1][:,:,1,1] = features[:,:,1,1]

println("\nCorrect label index: ", Int64(labels[:,1][1]))

forward(run_net)
println()
println("Label prob vector:")
println(run_net.output_blobs[:prob].data)

using MLBase
offset = -1  #my prediction vector is 1-based as opposed to the label
             #set which is 0-based.  So this offset matches them
prediction = classify(run_net.output_blobs[:prob].data) + offset
println("Prediction is: ", prediction[:][1])

#destroy the net
destroy(run_net)
shutdown(backend)

# Show the digit
A = features[:,:,1,1]
digit = convert(Image, A)
#default spatialorder is yx
digit["spatialorder"] = ["x", "y"]
digit



#### Run the net 10 times and show the images

In [1]:
"""
`input(prompt::AbstractString="")`

Read a string from STDIN. The trailing newline is stripped.

The prompt string, if given, is printed to standard output without a
trailing newline before reading input.
"""
function input(prompt::AbstractString="")
   print(prompt)
   return chomp(readline())
end

using Mocha

backend = CPUBackend()
init(backend)

#Create a memory data layer in the prediction model that we computation
#sequentially tie to the next image we want to predict on.
mem_data = MemoryDataLayer(name="data", tops=[:data], batch_size=1,
  data=Array[zeros(Float32, 28, 28, 1, 1)])

#Create the network with two conv/pooling layers and two fully
#connected inner product layers
conv_layer = ConvolutionLayer(name="conv1", n_filter=20, kernel=(5,5),
  bottoms=[:data], tops=[:conv1])
pool_layer = PoolingLayer(name="pool1", kernel=(2,2), stride=(2,2),
  bottoms=[:conv1], tops=[:pool1])
conv2_layer = ConvolutionLayer(name="conv2", n_filter=20, kernel=(5,5),
  bottoms=[:pool1], tops=[:conv2])
pool2_layer = PoolingLayer(name="pool2", kernel=(2,2), stride  = (2,2),
  bottoms=[:conv2], tops=[:pool2])
fc1_layer = InnerProductLayer(name="ip1", output_dim=500,
  neuron=Neurons.ReLU(), bottoms=[:pool2], tops=[:ip1])
fc2_layer = InnerProductLayer(name="ip2", output_dim=10,
  bottoms=[:ip1], tops=[:ip2])

#Instead of a SoftmaxLossLayer like the training script in this file we
#have a pure SoftmaxLayer in order to determine the probability of the 
#image compared against the different n classification options
softmax_layer = SoftmaxLayer(name="prob", tops=[:prob], bottoms=[:ip2])

#Build the network
common_layers = [conv_layer, pool_layer, conv2_layer, pool2_layer,
  fc1_layer, fc2_layer]
run_net = Net("imagenet", backend, [mem_data, common_layers..., softmax_layer])

#Load the latest snapshot from the training data
load_snapshot(run_net, "snapshots_bak/snapshot-010000.jld")

#Open the data source and load the features and labels
using HDF5, MLBase, ImageView, Images
fid = h5open("data/test.hdf5", "r")
features = read(fid, "data")
labels = read(fid, "label")[:] #put the labels into a single vector

offset = -1 #my prediction vector is 1-indexed as opposed to the label
            #set which is 0-indexed.  So this offset matches them

#load the initial image to get the handle to the canvas
A1 = features[:,:,1,1]
digit = convert(Image, A1)
digit["spatialorder"]=["x", "y"]
imgc, imgs = view(digit)
Tk.set_size(ImageView.toplevel(imgc), 200, 200)

#set up the loop
for i in 1:25
    #load the data into the layer
    get_layer(run_net, "data").data[1][:,:,1,1] = features[:,:,1,i]
    forward(run_net)
    prediction = classify(run_net.output_blobs[:prob].data) + offset
    prediction = prediction[:][1]
    gt = convert(Int64, labels[i])
    println("Sample $i is: $gt==>$prediction")
    
    #show the image
    A = features[:,:,1,i]
    digit = convert(Image, A)
    #default spatialorder is yx
    digit["spatialorder"] = ["x", "y"]
    view(imgc, digit)
    
    #pause for Enter
    input("Press the Enter key to continue:")
    #pause for 1/2 second
    #sleep(1.0)
    
end

println("******* TEST COMPLETE ********")

#destroy the net
Mocha.destroy(run_net)
Mocha.shutdown(backend)


Configuring Mocha...


 * CUDA       disabled by default
 * Native Ext disabled by default
Mocha configured, continue loading module...
DefaultBackend = Mocha.CPUBackend
11-Oct 19:53:03:INFO:root:Constructing net imagenet on Mocha.CPUBackend...
11-Oct 19:53:03:INFO:root:Topological sorting 8 layers...
11-Oct 19:53:03:INFO:root:Setup layers...
11-Oct 19:53:05:INFO:root:Network constructed!
11-Oct 19:53:08:INFO:root:Loading existing model from snapshots_bak/snapshot-010000.jld
11-Oct 19:53:11:DEBUG:root:Loading parameters for layer conv1
11-Oct 19:53:11:DEBUG:root:Loading parameters for layer conv2
11-Oct 19:53:11:DEBUG:root:Loading parameters for layer ip1
11-Oct 19:53:11:DEBUG:root:Loading parameters for layer ip2
Sample 1 is: 4==>4
Press the Enter key to continue:STDIN> 
Sample 2 is: 2==>2
Press the Enter key to continue:STDIN> 
Sample 3 is: 6==>6
Press the Enter key to continue:STDIN> 
Sample 4 is: 6==>6
Press the Enter key to continue:STDIN> 
Sample 5 is: 2==>2
Press the Enter key to continue:STDIN> 
Samp

LoadError: LoadError: UndefVarError: destroy not defined
while loading In[1], in expression starting on line 95